<a href="https://colab.research.google.com/github/Sudaiii/Proyecto-ML/blob/main/Entrega_02_Scrub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proyecto Final - Reconocimiento de Actividad Humana (HAR)


In [153]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

###Entrega 01 - OBTAIN

Incluir todo el código de la Etapa 01 - OBTAIN creada en la semana anterior

In [154]:
url = "https://drive.google.com/file/d/1Myzc7VaWS_9lpnm9smWMV_eNLPoLlzSL/view?usp=share_link"
url = "https://drive.google.com/uc?id=" + url.split('/')[-2]

wisdm = pd.read_csv(url, on_bad_lines="skip", header=None,)
wisdm.head()

,0,1,2,3,4,5
0,33,Jogging,49105962326000,-0.694638,12.680544,0.50395286;
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433;
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209;
3,33,Jogging,49106222305000,-0.612916,18.496431,3.0237172;
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164;


###Entrega 02 - SCRUB

Eliminar filas con datos faltantes

*Uno de los pasos iniciales en la limpieza de datos consiste en eliminar aquellas filas que contengan datos faltantes. Estas líneas generalmente no aportan información y pueden complicar la etapa de entrenamiento y validación de muchos modelos predictivos.*


In [155]:
wisdm.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    1
dtype: int64

In [156]:
wisdm = wisdm.dropna()
wisdm.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

Agregar al DataFrame los nombres de las columnas 

*En ocasiones al crear los data frames el sistema asigna nombres genéricos y correlativos a las columnas. Es recomendable renombrar las columnas por nombres significativos para favorecer el análisis posterior de los datos.*

In [157]:
wisdm.columns = ["user", "activity", "timestamp", "x-accel", "y-accel", "z-accel"]

Visualizar las 5 primeras filas del DataFrame (.head())

*Siempre es buena medida visualizar algunas líneas del dataframe para tener una idea concreta del contenido.*


In [158]:
wisdm.head()

,user,activity,timestamp,x-accel,y-accel,z-accel
0,33,Jogging,49105962326000,-0.694638,12.680544,0.50395286;
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433;
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209;
3,33,Jogging,49106222305000,-0.612916,18.496431,3.0237172;
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164;


Buscar datos nulos y eliminarlos si es que existen

In [159]:
# ¿De nuevo?

Visualizar los tipo de datos a los que pertenece cada columna (.info())

*Es necesario tener un detalle de cuantas variables están considerando y el tipo de datos que pertenecen para verificar si es posible utilizarla directamente para el modelo predictivo o si es necesario realizar una transformación del tipo de dato.*


In [160]:
wisdm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1086465 entries, 0 to 1086465
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   user       1086465 non-null  int64  
 1   activity   1086465 non-null  object 
 2   timestamp  1086465 non-null  int64  
 3   x-accel    1086465 non-null  float64
 4   y-accel    1086465 non-null  float64
 5   z-accel    1086465 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 58.0+ MB


Cambiar el tipo de datos de las variables predictoras a valores reales, para realizar la clasificación 

*Muchas de las variables contenidas en el DataFrame originalmente se encuentran en el tipo de dato 'object'. En ese formato las variables no pueden ser interpretadas como vectores. Recordemos que la mayoría de los modelos de Machine Learning estudiados operan en el espacio vectorial. Por esta razón las variables que formarán parte del modelo predictivo deben ser transformadas a un formato numérico como 'Float' o 'integer'.*

Para este proyecto en particular se pide transformar  al tipo de dato Float, las variables del acelerómetro en el eje $x$, $y$ y $z$



In [161]:
wisdm["z-accel"] = wisdm["z-accel"].str.replace(";", "").astype(float)

Imprimir los tipos de datos de las variables del DataFrame

*Se recomienda volver a ejecutar el comando info para verificar que efectivamente las variables cambiaron su tipo de dato y que ahora se pueden procesar con los modelos predictivos.*


In [162]:
wisdm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1086465 entries, 0 to 1086465
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   user       1086465 non-null  int64  
 1   activity   1086465 non-null  object 
 2   timestamp  1086465 non-null  int64  
 3   x-accel    1086465 non-null  float64
 4   y-accel    1086465 non-null  float64
 5   z-accel    1086465 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 58.0+ MB


Eliminar las columnas que no se utilizaran para la clasificación

*En este caso deberíamos conservar la actividad respectiva y las variables asociadas al acelerómetro*


In [163]:
wisdm_culled = wisdm.drop(columns=["user", "timestamp"])
wisdm_culled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1086465 entries, 0 to 1086465
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   activity  1086465 non-null  object 
 1   x-accel   1086465 non-null  float64
 2   y-accel   1086465 non-null  float64
 3   z-accel   1086465 non-null  float64
dtypes: float64(3), object(1)
memory usage: 41.4+ MB


Utilizar el comando *value_counts* para obtener un detalle de cuantas instancias se tienen por actividad. El objetivo consiste en identificar la presencia del balance o desbalance en las categorías a modelar


In [164]:
wisdm_culled["activity"].value_counts()

Walking       418393
Jogging       336445
Upstairs      122869
Downstairs    100425
Sitting        59939
Standing       48394
Name: activity, dtype: int64

Este proyecto considera datos con un desbalance de clases. Es necesario balancearlos para que los modelos operen de manera adecuada. Existen muchas maneras de realizar un balance, pero en este proyecto utilizaremos una estrategia sencilla e intuitiva. Simplemente debe identificar la actividad que contenga el número menor de instancias. Se debe realizar un muestreo para las otras actividades utilizando ese valor. De esta manera todas las clases tendrán la misma cantidad de instancias. 



In [165]:
# Se separan los datos de distintos valores
walking = wisdm_culled[wisdm_culled["activity"] == "Walking"]
jogging  = wisdm_culled[wisdm_culled["activity"] == "Jogging"]
upstairs = wisdm_culled[wisdm_culled["activity"] == "Upstairs"]
downstairs = wisdm_culled[wisdm_culled["activity"] == "Downstairs"]
sitting = wisdm_culled[wisdm_culled["activity"] == "Sitting"]
standing = wisdm_culled[wisdm_culled["activity"] == "Standing"]

# Para cada valor, se selecciona una cantidad equivalente 
# al numero de instancias "Standing"
walking = walking.head(len(standing))
jogging = jogging.head(len(standing))
upstairs = upstairs.head(len(standing))
downstairs = downstairs.head(len(standing))
sitting = sitting.head(len(standing))

# Se concatenan los dataframes reducidos
wisdm_culled_downsampled = pd.concat([walking, jogging, upstairs, downstairs, sitting, standing])

Establecer el número de instancias y dimensiones del data frame resultante *(shape)*


In [166]:
wisdm_culled_downsampled.shape

(290364, 4)

Volver a imprimir la cantidad de instancias por categoría para verificar que todo esté en orden. *(value_counts)*

In [167]:
wisdm_culled_downsampled["activity"].value_counts()

Walking       48394
Jogging       48394
Upstairs      48394
Downstairs    48394
Sitting       48394
Standing      48394
Name: activity, dtype: int64

Volver a visualizar las cinco primeras líneas del DataFrame para revisar que todo esté en orden *(head)*


In [168]:
wisdm_culled_downsampled.head()

,activity,x-accel,y-accel,z-accel
597,Walking,0.844462,8.008764,2.792171
598,Walking,1.116869,8.621680,3.786457
599,Walking,-0.503953,16.657684,1.307553
600,Walking,4.794363,10.760075,-1.184970
601,Walking,-0.040861,9.234595,-0.694638


Separar las variables predictoras (X) de las ctegorias o etiquetas (y)

In [169]:
X = wisdm_culled_downsampled[["x-accel", "y-accel", "z-accel"]]
y = wisdm_culled_downsampled[["activity"]]

Estandarizar los datos numericos (*StandardScaler)*

*Para el caso de la estandarización de los datos se pueden usar distintas técnicas, como la normalización 01, estandarización entre -1 y 1 o el denominado Z-Score. En este caso usaremos la función Standard Scaler de Python con sus valores por defecto.*


In [170]:
sc_x = StandardScaler()

X = sc_x.fit_transform(X)
X

array([[-0.03449447,  0.0769677 ,  0.2664932 ],
       [ 0.01727895,  0.18754729,  0.49575718],
       [-0.29077293,  1.63736749, -0.07583245],
       ...,
       [-1.06166167,  0.16559432, -0.58254798],
       [-1.04645694,  0.17281094, -0.62635849],
       [-1.13388413,  0.1259029 , -0.57101889]])

Como último paso debe reconstruir el dataframe pegando las columnas de los variables predicadores (X) y las categorías (y)


In [171]:
wisdm_reconstructed = pd.merge(y, pd.DataFrame(X, columns=["x-accel", "y-accel", "z-accel"]), left_index=True, right_index=True)
wisdm_reconstructed.head()

,activity,x-accel,y-accel,z-accel
597,Walking,-0.086268,0.386590,-0.173191
598,Walking,1.013917,0.782220,-0.474689
599,Walking,0.097528,0.740445,-0.747921
600,Walking,-0.383965,1.416209,-0.210878
601,Walking,-0.391731,-0.898589,-0.509236
